In [ ]:
def sigmoid(z):
    g = 1/(1+np.exp(-z))
    return g

In [ ]:
def comp_cost_logistic(X,y,w,b,lamda):
    m,n = X.shape
    
    z = np.dot(X, w) + b
    fwb = sigmoid(z)
    fwb_clipped = np.clip(fwb, 1e-15, 1 - 1e-15) 
    loss = (-y * np.log(fwb_clipped) - (1 - y) * np.log(1 - fwb_clipped))
    cost = np.sum(loss)/(2*m)
    
    cost_w = (lamda / (2 * m)) * np.sum(w**2)

    total_cost = cost+cost_w
    return total_cost

In [ ]:
def comp_gradient_logistic(X,y,w,b,lamda):
    m,n=X.shape
    z = np.dot(X, w) + b  
    fwb = sigmoid(z)
    error = fwb - y
    dj_dw = np.dot(X.T, error) / m
    dj_db = np.sum(error) / m
    reg_term = (lamda / m) * w
    dj_dw = dj_dw + reg_term
    return dj_dw,dj_db

In [ ]:
def gradient_descent_logistic(X,y,w_strt,b_strt,alpha,iters,lamda):
    m,n = X.shape
    w=w_strt
    b=b_strt
    J_data = []
    i=0
    while i<iters:
        dj_dw,dj_db = comp_gradient_logistic(X,y,w,b,lamda)
        w=w-alpha*dj_dw
        b=b-alpha*dj_db
        J_data.append(comp_cost_logistic(X,y,w,b,lamda))
        if(i%(iters//10)==0):
            print(f"Cost after {i}th iteration: {J_data[i]}")
        i+=1
    return w,b,J_data

In [ ]:
def predict(X, w, b):
    z = np.dot(X, w) + b
    fwb = sigmoid(z)
    y_pred = np.where(fwb >= 0.5, 1, 0)
    return y_pred

In [ ]:
def calculate_f1_score(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0

    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    if (precision + recall) == 0:
        return 0.0
    
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1


In [ ]:
def compute_class_weights(y):
    classes, counts = np.unique(y, return_counts=True)
    m = len(y)
    k = len(classes)

    weights = m / (k * counts)
    return weights


In [ ]:
def one_hot_encode(y,classes):
    class_to_idx = {c:i for i,c in enumerate(classes)}
    y_idx = np.array([class_to_idx[c] for c in y])
    Y_onehot = np.eye(len(classes))[y_idx]

    return Y_onehot

In [ ]:
def softmax(z):
    g = np.exp(z-np.max(z,axis=1,keepdims=True))
    return g/np.sum(g,axis=1,keepdims=True)

In [ ]:
def comp_cost_softmax(X,Y_onehot,W,b,lamda,class_weights):
    m=X.shape[0]
    z=np.dot(X,W)+b
    Y_pred=softmax(z)
    Y_pred = np.clip(Y_pred, 1e-15, 1 - 1e-15)
    cost=-np.sum(Y_onehot*class_weights*np.log(Y_pred))/m
    w_cost=(lamda/(2*m))*np.sum(W**2)
    total_cost=cost+w_cost
    return total_cost